# Init

In [1]:
%load_ext autoreload
%autoreload 2
from context import rmlib
import rmlib.rmtools as rm
from config import robotConfig


robot = rmlib.robot.Robot( robotConfig )

#Imports 
from time import sleep
import numpy as np
import rmlib
from rmlib.rmtools.assembly_trees import augment_RM

augment_RM( robot )

%store -r

####### LIBS #######################################################################################################################################

# Import Pytrees
import py_trees
from py_trees.tests import Timeout_Success
from py_trees.composites import Sequence , Selector

from assembly_trees import *


import math
from pmath import translate_pose , get_disance_between_poses , pose_components

from utils import is_matx_list
from math import radians

print( "Libs loaded!" )

Loaded: /home/nvidia/dev_rmstudio/rmlib/rmlib/rmtools Exists?: True
Loaded: /home/nvidia/dev_rmstudio/rmlib Exists?: True
Setting up robot please wait...
my_components: {'hand_config': {'class_name': 'SmartHand', 'module_name': 'hands.smarthand', 'finger_width_inner': 0.0, 'finger_length': 0.0415, 'finger_width_outer': 0.015, 'finger_depth': 0.014}, 'cam_config': {'class_name': 'RealSense', 'ci_cam_offset': [0, 0, 0], 'module_name': 'cameras.realsense', 'camera_model': 'd410', 'pc_cam_offset': [0, 0, 0]}, 'arm_config': {'max_linear_speed': 0.25, 'class_name': 'UR5', 'ip_address': '192.168.0.7', 'max_linear_accel': 1.2, 'xmlrpc_port': '8003', 'max_joint_accel': 1.4, 'max_joint_speed': 1.05, 'module_name': 'arms.ur5', 'default_linear_accel': 0.8, 'default_linear_speed': 0.1, 'default_joint_accel': 0.8, 'default_joint_speed': 0.7}, 'ft_config': {'class_name': 'OptoForce', 'ip_address': '192.168.0.3', 'module_name': 'sensors.ftsensor_optoforce'}}
active_componetns: {'hand': ['hand_config']

# Global Vars

In [2]:
DATA_RATE  = 50
SLEEP_TIME =  5.0

# Tilt Insert: Shaft Recording

In [ ]:
def get_shaft_insert_tree():
    rootNode = Sequence_Recorder(  memory = 0 , dataKey = "test_DATA" , flagKey = "test_FLAG" , 
                                   outFileGenFunc = get_XML_outfile_namer( pathPrefix = "output/" , namePrefix = "SHAFT-TLT_" )  )

    recordBehav = Record_Classify( recordFlagKey = "test_FLAG" , dataKey = "test_DATA" , recordHz = DATA_RATE , ctrl = robot )


    insertTree = Sequence( memory = 1 )

    goBehav = Jog_Safe( shaftGraspPose , 
                        zSAFE = 0.100 ,  
                        hover = 1 ,  
                        ctrl  = robot  )

    graspBehav = Grasp_at_Pose( shaftGraspPose, 0.015 ,
                                zApproach=0.050 , zClose=0.0 , wdthNarrow = 0.030 , maxIter=5, zFree=0.100 , 
                                ctrl = robot )

    jogBehav = Jog_Safe( shaftPreInsertPose , 
                        zSAFE = 0.100 ,  
                        hover = 0 ,  
                        ctrl  = robot  )


    tiltBehav = Tilt_Insert( shaftPreInsertPose ,
                              tilt_angle_deg = 7.25 , part_offset = 0.016 , dia = 0.015 , 
                              touch_force = 0.6 , insert_force = 1.5 , max_movement = 0.045 , biasWrist = 1 , offset_dir = np.array([1.0,0.0,0.0]), 
                              posnMargin = 0.003 , orntMargin = 2.0/180 , finalInsertHandZ = fatPegFullInsertZ , recordFlagKey = "test_FLAG" , ctrl = robot )

    dropBehav = Set_Fingers( openState = 1.0 , ctrl = robot )    

    insertTree.add_children( [ goBehav , graspBehav , jogBehav , tiltBehav , dropBehav ] )

    rootNode.add_children( [ recordBehav , insertTree ] )
    
    return rootNode

# Tilt Insert Pipeline

In [ ]:
N = 100
from time import sleep

for i in range( N ):
    
    try:
        
        print( "Iteration" , i+1 , "of" , N )
        
        # 1. Reset tree
        rootNode = get_shaft_insert_tree()
        
        # 2. Run tree and generate recording
        run_BT_until_done( rootNode , N = 1000 , tickPause = 0.25 , breakOnFailure = 1 , breakOnSuccess = 1 , Nverb = 0 )
        
        # 3. Release
        robot.hand.release()
        
        # 4. Deactivate motors
        robot.hand.deactivate_motors()
        
        # 5. Move to safe position
        robot.arm.move( safeCenterPose )
        
        # 6. Sleep for human reset
        sleep( SLEEP_TIME )

    except KeyboardInterrupt:
        print( "Pipeline was STOPPED by user at iteration" , i+1 )
        break

# Spiral Insert Process

In [3]:
class Tap_Down( py_trees.composites.Sequence ):
    """ For dislodging vertically and pushing down """
    # SEE Push_and_Twist_to_Level for inspiration
    # Push_and_Twist_to_Level is broken
    
    def __init__( self , goalZ, allowedF = 5, tap_force = 1.0 , zRetract = 0.040, tapWidth = 0.020, 
                 plungeDistance = 0.070, descendSpeed=0.0125 , zMargin = 0.005, ctrl = None ):
        """ Construct the subtree """
    
        super().__init__( name = "Tap_Down" , memory = True )

        # ~~ Add Nodes ~~
        
        # 0. Open the hand
        
        self.add_child(Set_Fingers( openState = 1.0 , ctrl = ctrl ))
        
        # 1. Move up
        
        #self.add_child(Move_Arm_Relative( translation = [0.0,0.0,zRetract] , ctrl = ctrl ))
        
        # 2. Narrow fingers to distance
                       
        self.add_child(Set_Fingers( openState = tapWidth , ctrl = ctrl ))
                       
        
        # 3. Tap Loop
        
        #Tap_Routine = py_trees.composites.Selector()
        Tap_Routine = py_trees.composites.Sequence( memory = 1 )
        
        
        Tap_Routine.add_child()
        # 3.a Move to contact
        
        Tap_Routine.add_child()
        
#         Tap_Routine.add_child(py_trees.decorators.SuccessIsFailure(Move_to_Contact( relMove = [ 0 , 0 , -plungeDistance ] , Fmag = tap_force , 
#                                               speed = descendSpeed , biasWrist = 1 , ctrl = ctrl )))
        
        # 3.b Check Z
                              
        Tap_Routine.add_child(   )
        
        # 3.c Move to saved pose
        
#         Tap_Routine.add_child(py_trees.decorators.SuccessIsFailure(Move_Arm_Relative( translation = [ 0 , 0 , 1.7*plungeDistance ] , 
#                                                                                      speed = descendSpeed,ctrl = ctrl )))
        
        self.add_child(Run_to_X_Failures_DECO(Tap_Routine, X_allowedFails = allowedF))
        
        # 4 Move to saved pose
        
        self.add_child(Move_Arm_Relative( translation = [ 0 , 0 , plungeDistance ] , speed = descendSpeed, ctrl = ctrl ))
        

In [ ]:
rootNode = Tap_Down(  goalZ = 0.0472, allowedF = 5, tap_force = 7.0 , zRetract = 0.040, tapWidth = 0.020, 
                 plungeDistance = 0.10, descendSpeed=0.025 , zMargin = 0.005, ctrl = robot )

In [6]:
from math import radians

class TwistToggleRelGen:
    """ Generator for a toggled twist motion, Relative """
    
    def __init__( self , twist_deg = 10 , doubleSided = 1 , N = None , ctrl = None ):
        """ Set vars for the twist operation """
        self.ctrl     = ctrl
        self.twistRad = radians( twist_deg )
        self.dbblSdd  = doubleSided
        self.N        = N
        self.factor   = 1.0
        self.i        = 1
        
    def __call__( self ):
        """ Generate the next pose """
        # 1. Get the hand pose
        pose = self.ctrl.arm.get_tcp_pose()
        # 2. Calc the next pose
        turnRad = 0
        if self.dbblSdd:
            print( "double sided" )
            
            if self.N != None: 
                print( "specified N" )
                # B. If double sided && If iter-limited && if the last iteration, Then turn by hald
                if self.i == 1:
                    print( "the first iteration" )
                    turnRad = self.twistRad * 0.5
                elif self.N == self.i:
                    print( "the last iteration" )
                    turnRad = self.twistRad * 0.5
                # B. If double sided && If iter-limited && if surpassed last iteration, Then do not turn
                elif self.N < self.i:
                    print( "too many iteration" )
                    turnRad = 0.0
                # C. Else turn by full measure
                else:
                    print( "iteration:" , self.i )
                    turnRad = self.twistRad
            else:
                print( "no N given" )
                # A. If double sided && If the first iteration, Then turn by half
                if self.i == 1:
                    print( "the first iteration" )
                    turnRad = self.twistRad * 0.5
                else:
                    turnRad = self.twistRad
            
        # D. Else is not double sided, turn by full measure
        else:
            print( "NOT double sided" )
            turnRad = self.twistRad
        # 3. Rotate the pose
        turnRad *= self.factor
        print( "turnRad =" , turnRad )
        pose    = rotate_pose( pose , [ 0.0 , 0.0 , turnRad ] , dir_pose = 'self' )
        # 4. Reverse the direction
        self.factor *= -1.0
        # 5. Incrment `i`
        self.i += 1
        # 6. Return the pose
        return pose

In [7]:
class Tamp_Alternating( py_trees.composites.Sequence ):
    """ Attempt to remedy a jammed insertion """
    
    def __init__( self , zGoal , zMargin = 0.005 , tap_force = 3.0 , plungeDistance = 0.070 , tapWidth = 0.020 ,
                  twist_deg = 10 , doubleSided = 1 , 
                  descendSpeed = 0.0125 , Ntries = 6 , TzStop = 1.8 , ctrl = None ):
        """ Build the subtree """
        super().__init__( name = "Tamp_Alternating" , memory = 1 )
        
        # -1. set finger width
        self.add_child(  Set_Fingers( openState = tapWidth , ctrl = ctrl )  )
        
        # 0. Instantiate tamp subtree
        pushTwist = py_trees.composites.Sequence( name = "Push_and_Twist_Seq" , memory = 1 )

        # 1. Retract
        pushTwist.add_child(  Move_Arm_Relative( translation = [ 0 , 0 , 0.5*plungeDistance ] , speed = descendSpeed , ctrl = ctrl )  )
        
        # 2. Twist
        def cond_Tz( *args ):
            """ Stop at the prescribed wrist torque """
            wrench = ctrl.ft.get_wrist_force()
            if abs( wrench[5] ) > TzStop:
                return 1
            else:
                return 0
        
        pushTwist.add_child(  Move_Rule_w_Stop_Cond( 
            TwistToggleRelGen( twist_deg = twist_deg , doubleSided = doubleSided , N = Ntries , ctrl = ctrl ) , 
            stop_cond    = cond_Tz , 
            condSuccess  = False   , 
            relativeRule = 0       , 
            ctrl         = ctrl 
        )  )
        
        # 3. Tamp
        pushTwist.add_child(  Move_to_Contact( relMove = [ 0 , 0 , -plungeDistance ] , Fmag = tap_force , speed = descendSpeed , biasWrist = 1 , ctrl = ctrl )  )
        
        # 4. Check z level
        pushTwist.add_child(  At_Z_Level_COND( zGoal , margin = zMargin , ctrl = ctrl )   )
        
        # 5. In a loop
        self.add_child(  
            Run_to_X_Failures_DECO(  
                pushTwist ,
                X_allowedFails = Ntries
            )
        )

In [8]:
rootNode = Tamp_Alternating( zGoal = 0.0472 , zMargin = 0.005 , tap_force = 3.0 , plungeDistance = 0.070 , tapWidth = 0.020 ,
                             twist_deg = 90 , doubleSided = 0 , 
                             descendSpeed = 0.0125 , Ntries = 6 , TzStop = 1.8 , ctrl = robot )

In [9]:
run_BT_until_done( rootNode , N = 1000 , tickPause = 0.25 , breakOnFailure = 1 , breakOnSuccess = 1 , Nverb = 10 )

Root node Tamp_Alternating status: Status.RUNNING
Root node Tamp_Alternating status: Status.RUNNING
Root node Tamp_Alternating status: Status.RUNNING
Root node Tamp_Alternating status: Status.RUNNING
Root node Tamp_Alternating status: Status.RUNNING
Root node Tamp_Alternating status: Status.RUNNING
Root node Tamp_Alternating status: Status.RUNNING
Root node Tamp_Alternating status: Status.RUNNING
Root node Tamp_Alternating status: Status.RUNNING

--------- Tick 10 ---------



[-] Tamp_Alternating [*]
    --> Set_Fingers [o]
    -^- Run_to_X_Failures_DECO [*]
        [-] Push_and_Twist_Seq [*]
            --> Move_Arm_Relative [*]
            --> Move_Rule_w_Stop_Cond [-]
            --> Move_to_Contact [-]
            --> At_Z_Level_COND [-]

Root node Tamp_Alternating status: Status.RUNNING
Root node Tamp_Alternating status: Status.RUNNING
Root node Tamp_Alternating status: Status.RUNNING
Root node Tamp_Alternating status: Status.RUNNING
Root node Tamp_Alternating status: Status.RUNN

In [5]:
from assembly_trees import _DUMMYPOSE
import py_trees

def get_spiral_test_tree():
    rootNode = Sequence_Recorder(  memory = 0 , dataKey = "test_DATA" , flagKey = "test_FLAG" , 
                                   outFileGenFunc = get_XML_outfile_namer( pathPrefix = "output/" , namePrefix = "PULLEY-SPR_" )  )

    recordBehav = Record_Classify( recordFlagKey = "test_FLAG" , dataKey = "test_DATA" , recordHz = 50 , ctrl = robot )


    insertTree = Sequence( memory = 1 )

    goBehav = Jog_Safe( wheelGraspPose , 
                        zSAFE = 0.100 ,  
                        hover = 1 ,  
                        ctrl  = robot  )

    graspBehav = Grasp_at_Pose( wheelGraspPose, 0.026 ,
                                zApproach=0.050 , zClose=0.0 , wdthNarrow = 0.040 , maxIter=5, zFree=0.100 , 
                                ctrl = robot )

    jogBehav = Jog_Safe( wheelPreOffsetXPose , #wheelPreOffsetYPose,  # wheelPreInsertPose , 
                         zSAFE = 0.100 ,  
                         hover = 0 ,  
                         ctrl  = robot  )

    # 1.8 # Lateral torque caused it to bind
    insertBehav = Spiral_Insert( wheelPreOffsetXPose , #wheelPreOffsetYPose , # wheelPreInsertPose ,
                                 touch_force = 0.8 , drop_force = 0.6 , insert_force = 4 , max_movement = 0.050 , 
                                 lateralStopTorque = 0.45 , pushbackF = 20.0 , spiralSpeed = 0.002 , descendSpeed = 0.00625 , biasWrist = 1 ,
                                 degrees_to_step = 10 , startRadius = 0.0005 , stepSize = 0.00025 ,  max_radius=0.060 ,
                                 posnMargin = 0.003 , orntMargin = 2.0/180 , finalInsertHandZ = None , 
                                 reliefStep_m = 0.00025 , reliefMargin_Nm = 0.04 , reliefN = 10 , # This line tuned 2020-06-18
                                 ctrl = robot )

    dropBehav = Set_Fingers( openState = 1.0 , ctrl = robot )    

    insertTree.add_children( [ goBehav , graspBehav , jogBehav , insertBehav , dropBehav ] )

    rootNode.add_children( [ recordBehav , insertTree ] )
    
    return rootNode

    

# Spiral Insert: Recording Pipeline

In [6]:
N = 3
from time import sleep

for i in range( N ):
    
    try:
        
        print( "Iteration" , i+1 , "of" , N )
        
        # 1. Reset tree
        rootNode = get_spiral_test_tree()
        
        # 2. Run tree and generate recording
        run_BT_until_done( rootNode , N = 1000 , tickPause = 0.25 , breakOnFailure = 1 , breakOnSuccess = 1 , Nverb = 0 )
        
        # 3. Release
        robot.hand.release()
        
        # 4. Deactivate motors
        robot.hand.deactivate_motors()
        
        # 5. Move to safe position
        robot.arm.move( safeCenterPose )
        
        # 6. Sleep for human reset
        sleep( SLEEP_TIME )

    except KeyboardInterrupt:
        print( "Pipeline was STOPPED by user at iteration" , i+1 )
        break
        

Iteration 1 of 3
COND_At_TCP_Pose , SUCCEEDED: Reached the pose
Root node Sequence_Recorder failed!


Simulation completed! with status rootNode.status 


about to WRITE to output/PULLEY-SPR_2020-06-19_03-40-57.xml, writing COMPLETE
Iteration 2 of 3
COND_At_TCP_Pose , SUCCEEDED: Reached the pose
Root node Sequence_Recorder failed!


Simulation completed! with status rootNode.status 


about to WRITE to output/PULLEY-SPR_2020-06-19_03-41-22.xml, writing COMPLETE
Iteration 3 of 3
COND_At_TCP_Pose , SUCCEEDED: Reached the pose
Root node Sequence_Recorder failed!


Simulation completed! with status rootNode.status 


about to WRITE to output/PULLEY-SPR_2020-06-19_03-41-48.xml, writing COMPLETE


# Common Functions

In [5]:
robot.arm.align_tcp( robot.arm )

True

In [6]:
robot.hand.grip()

True

In [7]:
robot.hand.release()

False

In [9]:
robot.hand.set_finger_width( 0.020 ) 

True

In [7]:
robot.arm.move( wheelPreInsertPose )

In [8]:
robot.arm.get_tcp_pose()

array([[-0.99836891, -0.05700227,  0.00320382, -0.05069704],
       [-0.05694098,  0.99823702,  0.01675048, -0.53282457],
       [-0.00415299,  0.01654073, -0.99985457,  0.04725475],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [14]:
robot.hand.deactivate_motors()

1

# Setup

In [ ]:
# wheelGraspPose = robot.arm.get_tcp_pose()
# %store wheelGraspPose

In [9]:
# wheelPreInsertPose = robot.arm.get_tcp_pose()
# %store wheelPreInsertPose

Stored 'wheelPreInsertPose' (ndarray)


In [ ]:
# wheelPostInsertPose = robot.arm.get_tcp_pose()
# wheelPostInsertZ = wheelPostInsertPose[2,3]
# %store wheelPostInsertPose
# %store wheelPostInsertZ

In [35]:
# wheelPreOffsetYPose = robot.arm.get_tcp_pose()
# %store wheelPreOffsetYPose

Stored 'wheelPreOffsetYPose' (ndarray)


In [8]:
# wheelPreOffsetXPose = robot.arm.get_tcp_pose()
# %store wheelPreOffsetXPose

Stored 'wheelPreOffsetXPose' (ndarray)


In [ ]:
# shaftGraspPose = robot.arm.get_tcp_pose()
# %store shaftGraspPose

In [ ]:
# shaftPreInsertPose = robot.arm.get_tcp_pose()
# %store shaftPreInsertPose

In [ ]:
# safeCenterPose = robot.arm.get_tcp_pose()
# %store safeCenterPose

# TEST: Relieve Wrist Torque
1. Put the wrist in a slighly torqued place

In [ ]:
py_trees.logging.level = py_trees.logging.Level.INFO

rootNode =  Minimize_Wrist_Torque_XY( numTrials = 10 , marginT=0.10 , moveStep=0.002  , ctrl = robot )




In [ ]:
robot.ft.bias_wrist_force()

APPLY TORQUE HERE

In [ ]:
print( "FT before the behavior:" , robot.ft.get_wrist_force() )

run_BT_until_done( rootNode , N = 1000 , tickPause = 0.25 , breakOnFailure = 1 , breakOnSuccess = 1 )

print( "FT After the behavior:" , robot.ft.get_wrist_force() )